https://keras.io/preprocessing/image/

# TensorFlow 2.0

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Hyperparameter Tunning

In [2]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.7

input_shape = (32, 32, 3)
num_classes = 10

## Preprocess

In [3]:
import pandas as pd

In [4]:
train_df = pd.read_csv('train_dataset.csv')
test_df = pd.read_csv('test_dataset.csv')

In [5]:
test_df.head()

,name,class
0,dataset/cifar/test\0_cat.png,cat
1,dataset/cifar/test\1000_dog.png,dog
2,dataset/cifar/test\1001_airplane.png,airplane
3,dataset/cifar/test\1002_ship.png,ship
4,dataset/cifar/test\1003_deer.png,deer


In [6]:
train_datagen = ImageDataGenerator(
    rescale=1./255.,
    width_shift_range=0.3,
    zoom_range=0.2,
    horizontal_flip=True,
)

In [7]:
test_datagen = ImageDataGenerator(
    rescale=1./255.,
)

In [8]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='name',
    y_col='class',
    target_size=input_shape[:2],
    batch_size=batch_size,
)

Found 50000 validated image filenames belonging to 10 classes.


In [9]:
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='name',
    y_col='class',
    target_size=input_shape[:2],
    batch_size=batch_size,
)

Found 10000 validated image filenames belonging to 10 classes.


In [10]:
test_generator

## Build Model

In [11]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [12]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

## Training

In [13]:
batch_size

32

In [14]:
len(train_df) // 32

1562

In [15]:
len(train_generator)

1563

In [16]:
len(test_generator)

313

In [17]:
model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=test_generator,
    validation_steps=len(test_generator),
)

C:\Users\LGCNS\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
1563/1563 [==============================] - 225s 141ms/step - loss: 2.0698 - accuracy: 0.2219 - val_loss: 1.7146 - val_accuracy: 0.3949
Epoch 2/10
1563/1563 [==============================] - 211s 135ms/step - loss: 1.7182 - accuracy: 0.3612 - val_loss: 1.4760 - val_accuracy: 0.4475
Epoch 3/10
1563/1563 [==============================] - 240s 153ms/step - loss: 1.6374 - accuracy: 0.3975 - val_loss: 1.4169 - val_accuracy: 0.4688
Epoch 4/10
1563/1563 [==============================] - 172s 110ms/step - loss: 1.5868 - accuracy: 0.4157 - val_loss: 1.4046 - val_accuracy: 0.4845
Epoch 5/10
1563/1563 [==============================] - 161s 103ms/step - loss: 1.5573 - accuracy: 0.4309 - val_loss: 1.3711 - val_accuracy: 0.5018
Epoch 6/10
1563/1563 [==============================] - 222s 142ms/step - loss: 1.5296 - accuracy: 0.4444 - val_loss: 1.3551 - val_accuracy: 0.4934
Epoch 7/10
1563/1563 [==============================] - 207s 133ms/step - loss: 1.5019 - accuracy: 0.4576 - val_